In [1]:
import pandas as pd

import random
import re
import pickle
import nltk

In [2]:
dataset = pd.read_csv("dataset/review.csv",sep="\t")
#dataset.drop(columns= "Unnamed: 0",inplace = True)
dataset.head(3)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,timestamp,IP Address
0,US,20422322,R8MEA6IGAHO0B,B00MC4CED8,82850235,BlackVue DR600GW-PMP,Mobile_Electronics,5,0,0,N,Y,Very Happy!,"As advertised. Everything works perfectly, I'm...",2015-08-31,1440987567,193.93.167.87
1,US,40835037,R31LOQ8JGLPRLK,B00OQMFG1Q,82850235,GENSSI GSM / GPS Two Way Smart Phone Car Alarm...,Mobile_Electronics,5,0,1,N,Y,five star,it's great,2015-08-31,1441001998,193.93.167.87
2,US,51469641,R2Y0MM9YE6OP3P,B00QERR5CY,82850235,iXCC Multi pack Lightning cable,Mobile_Electronics,5,0,0,N,Y,great cables,These work great and fit my life proof case fo...,2015-08-31,1440959483,193.93.167.87


In [3]:
import pandas as pd

import random
import re
import pickle5 as pickle
import nltk
from sklearn.linear_model import LogisticRegression

from nltk.corpus import stopwords
from sklearn.datasets import load_files


with open("pickle_files/classifier.pickle","rb") as f:
    clf = pickle.load(f)

with open("pickle_files/TfidfModel.pickle","rb") as f:
    tfidf = pickle.load(f)


def getSentiment(text):

    text = str(text)
    text = text.lower()
    text = re.sub(r"that's","that is",text)
    text = re.sub(r"there's","there is",text)
    text = re.sub(r"what's","what is",text)
    text = re.sub(r"where's","where is",text)
    text = re.sub(r"it's","it is",text)
    text = re.sub(r"who's","who is",text)
    text = re.sub(r"i'm","i am",text)
    text = re.sub(r"she's","she is",text)
    text = re.sub(r"he's","he is",text)
    text = re.sub(r"they're","they are",text)
    text = re.sub(r"who're","who are",text)
    text = re.sub(r"ain't","am not",text)
    text = re.sub(r"wouldn't","would not",text)
    text = re.sub(r"shouldn't","should not",text)
    text = re.sub(r"can't","can not",text)
    text = re.sub(r"couldn't","could not",text)
    text = re.sub(r"won't","will not",text)
    
    text = re.sub(r"\W"," ",text)
    text = re.sub(r"\d"," ",text)
    text = re.sub(r"\s+[a-z]\s+"," ",text)
    text = re.sub(r"^[a-z]\s+"," ",text)    
    text = re.sub(r"\s+[a-z]$"," ",text)    
    text = re.sub(r"\s+"," ",text)    
    
    sent = clf.predict(tfidf.transform([text]).toarray())
    
    return sent[0]


##  Reviews which have dual view

In [4]:
##Different sentiment in review headline and review body

remove_reviews = []
# stores the list of review_id of fake reviews

for i in range(len(dataset)):
    #iterate through the whole dataset
    
        if( getSentiment( dataset["review_headline"][i] ) != getSentiment( dataset["review_body"][i] ) ):
            # checking if the sentiment of the body and the headline are not same
            
            remove_reviews.append(dataset["review_id"][i])
            # append review_id to the list of fake reviews.

In [5]:
len(remove_reviews)

42

In [6]:
print(remove_reviews)

['R31LOQ8JGLPRLK', 'R31LOQ8JGLPRLK', 'R3RRXU2R23NMQ9', 'RBEMQ29WJBHYG', 'R19VVIUT4BZCMT', 'RLIL6S9OGM3YA', 'R12UBZT87UX3AP', 'R2K0YQPV5W2SJ1', 'R104MY31OKNGXH', 'R3EQYRGIIJQBHT', 'R32MW6D6CXKCFW', 'R19C76D4LWB6CM', 'R5I2YBT9J85FF', 'R28UEIZZYTOC6N', 'R36A0KHN48CHBP', 'R3PG8VI81UURO5', 'R2F2BCP1CC8VKZ', 'R174DGBVW0UWAJ', 'R2074W1A28UUU6', 'R1M8PKOFRU2DZ', 'R2RWA9DSTPVRCI', 'R3B3E2E0123J3F', 'R2W7SNCOCHAVOK', 'R2EVA14XVX2MH3', 'R2DWIADFBQ3IYG', 'R2VMBHZ9XTC0GQ', 'R1L23IFB85ESH6', 'RAZX9ZCFPGCKO', 'R4JILH6CA3ZWE', 'RBKUGHHNXVYG0', 'RHK9SLC4KVI7E', 'R2O0LZSPMME6US', 'R3GGP4OW20EAGY', 'R1OLWVLQX55BH0', 'R2K2WK38XR5FKZ', 'RGAH8RF1K8QUI', 'R2JK5Y8D5MXAGP', 'R2SAHVFOHVRMS8', 'RIXC8QXB2OESY', 'R2EYF5O4W313NW', 'RP9SQJGGAJR7E', 'R2SXHHN0P24XX2']


## Reviews in which same user promoting or demoting a particular brand

In [7]:
# Users which are posting either all positive or negative reviews on different products of same brand
import pandas as pd 
dataset = pd.read_csv("dataset/review.csv",sep="\t")


customers = dataset.groupby("customer_id")
# groups dataset by customers

customer_list = dataset["customer_id"].unique()
#list of unique customers
 
size = len(customer_list.tolist())
#size of total unique customers

for i in range(size):
    # iterate through all the customers
    
    brand_df = customers.get_group(customer_list[i])    
    # Dataframe for each customers
    
    brands = brand_df.groupby("product_parent")
    # groups reviews of each customers by brand
    
    brands_list = brand_df["product_parent"].unique()
    # unique list of brands for each customers reviews
    
    no_of_brands = len(brands_list.tolist())
    # no. of brands for which reviews had been written by the customer
    
    for j in range(no_of_brands):
        # iterate through all the brands
        
        product_df = brands.get_group(brands_list[j])
        # Dataframe of products for a brand for which a customer had written reviews
        
        no_of_products = len(product_df["product_id"])
        # no of products
        
        if no_of_products<=2:
            # it will filter the products which are less than 2 for a brand
            continue
            
        indices = product_df.index.values.tolist()
        # index of the dataframe of the products of each brand for each customers
        
        sentiment = getSentiment(product_df["review_body"][indices[0]])
        # sentiment of the review of the first product
        
        isSameSentiment = True
        
        #discarding those cases in which we have only less than 3 reviews on same brand
        if(no_of_products<4):
            continue
        
        for k in range(1,no_of_products):
            # iterate through all the products
            
            text = str(product_df["review_body"][indices[k]])
            # review of each product
            
            if getSentiment(text)!=sentiment :
                # if sentiment is different than discard it
                isSameSentiment = False
                break;
                
        if(isSameSentiment):
            # If sentiments of all the products of same brand by a customer is same, 
            # Append customer_id to blocked users list
            
            remove_reviews.append(customer_list[i])
            break
        
    

In [8]:
len(remove_reviews)

42

In [9]:
print(remove_reviews)

['R31LOQ8JGLPRLK', 'R31LOQ8JGLPRLK', 'R3RRXU2R23NMQ9', 'RBEMQ29WJBHYG', 'R19VVIUT4BZCMT', 'RLIL6S9OGM3YA', 'R12UBZT87UX3AP', 'R2K0YQPV5W2SJ1', 'R104MY31OKNGXH', 'R3EQYRGIIJQBHT', 'R32MW6D6CXKCFW', 'R19C76D4LWB6CM', 'R5I2YBT9J85FF', 'R28UEIZZYTOC6N', 'R36A0KHN48CHBP', 'R3PG8VI81UURO5', 'R2F2BCP1CC8VKZ', 'R174DGBVW0UWAJ', 'R2074W1A28UUU6', 'R1M8PKOFRU2DZ', 'R2RWA9DSTPVRCI', 'R3B3E2E0123J3F', 'R2W7SNCOCHAVOK', 'R2EVA14XVX2MH3', 'R2DWIADFBQ3IYG', 'R2VMBHZ9XTC0GQ', 'R1L23IFB85ESH6', 'RAZX9ZCFPGCKO', 'R4JILH6CA3ZWE', 'RBKUGHHNXVYG0', 'RHK9SLC4KVI7E', 'R2O0LZSPMME6US', 'R3GGP4OW20EAGY', 'R1OLWVLQX55BH0', 'R2K2WK38XR5FKZ', 'RGAH8RF1K8QUI', 'R2JK5Y8D5MXAGP', 'R2SAHVFOHVRMS8', 'RIXC8QXB2OESY', 'R2EYF5O4W313NW', 'RP9SQJGGAJR7E', 'R2SXHHN0P24XX2']


## Reviews in which person from same IP Address promoting or demoting a particular brand

In [10]:
#Reviews from same IP either all positive or negative reviews on different products of same brand


ip = dataset.groupby("IP Address")
#grouping the dataset by ip address

ip_list = dataset["IP Address"].unique()
#stores the list pf unique ip addresses

remove_ip = []
#stores the list of ip address from where reviews have been written.

size = len(ip_list.tolist())
#stores the size of the total unique ip addresses

for i in range(size):
    # iterate through all the ip addresses
    
    brand_df = ip.get_group(ip_list[i])
    # Dataframe of brands for which reviews have been written from the same ip address
    
    brands = brand_df.groupby("product_parent")
    # grouping the products of the same brands for each ip addresses
    
    brands_list = brand_df["product_parent"].unique()
    #list of unique brands for each ip addresses
    
    no_of_brands = len(brands_list.tolist())
    # total no. of brands
    
    for j in range(no_of_brands):
        # iterate through all the brands
        
        product_df = brands.get_group(brands_list[j])
        # Dataframe of the products of each brand of each products
        
        no_of_products = len(product_df["product_id"])
        # no of products of each brand for each ip addresses
        
        if no_of_products<=2:
            # filter the reviews of the brandswith less than 3 reviews
            break
        
        indices = product_df.index.tolist()
        # indices of dataframe of products of each brand for each customers
        
        sentiment = getSentiment(product_df["review_body"][ indices[0] ])
        # sentiment of review of first product of each brand
                
        isSameSentiment = True
        
        for k in range(1,no_of_products):
            # iterate through all the reviews
            
            text = str(product_df["review_body"][indices[k]])
            # reviews of each product
            
            if getSentiment(text)!=sentiment :
                # if sentiment of 2 products of same brand are not same 
                # then check the next brand
                isSameSentiment = False
                break;
                
        if(isSameSentiment):
            # if all the sentiments are same , append ip to blocked list
            remove_ip.append(ip_list[i])
            

In [11]:
remove_ip

['193.93.167.87']

##  Reviews which are posted as flood by same user all the reviews are either positive or negative.

In [12]:
dataset.sort_values("customer_id",inplace=True)

In [13]:
remove_reviews = []  # Initialize list to store review IDs to be removed

customer_group = dataset.groupby("customer_id")
customer_group_list = dataset["customer_id"].unique().tolist()

for i in range(len(customer_group_list)):
    customer_reviews = customer_group.get_group(customer_group_list[i])
    dates_list = customer_reviews["review_date"].dropna().unique().tolist()  # Filter out NaN values
    reviews_by_date = customer_reviews.groupby("review_date")

    for j in range(len(dates_list)):
        reviews_by_date_for_pos = []
        reviews_by_date_for_neg = []

        df = reviews_by_date.get_group(dates_list[j])
        indices = df.index.tolist()

        for k in range(len(df)):
            text = df["review_body"][indices[k]]
            if getSentiment(text) == 0:
                reviews_by_date_for_neg.append(df["review_id"][indices[k]])
            else:
                reviews_by_date_for_pos.append(df["review_id"][indices[k]])

        if len(reviews_by_date_for_pos) > 3:
            remove_reviews.extend(reviews_by_date_for_pos)

        if len(reviews_by_date_for_neg) > 3:
            remove_reviews.extend(reviews_by_date_for_neg)


In [14]:
len(remove_reviews)

0

In [15]:
print(remove_reviews)

[]


##  Reviews which are posted as flood by same person from same IP Address

In [16]:
remove_reviews = []  # Initialize list to store review IDs to be removed

ip_group = dataset.groupby("IP Address")
ip_list = dataset["IP Address"].unique().tolist()

for i in range(len(ip_list)):
    reviews = ip_group.get_group(ip_list[i])
    dates_list = reviews["review_date"].dropna().unique().tolist()  # Filter out NaN values
    reviews_by_date = reviews.groupby("review_date")

    for j in range(len(dates_list)):
        reviews_by_date_for_pos = []
        reviews_by_date_for_neg = []

        try:
            reviews_for_each_day = reviews_by_date.get_group(dates_list[j])
        except KeyError:
            continue  # Skip to the next date if the current date key is not found
        
        indices = reviews_for_each_day.index.tolist()

        for k in range(len(reviews_for_each_day)):
            text = reviews_for_each_day["review_body"][indices[k]]
            if getSentiment(text) == 0:
                reviews_by_date_for_neg.append(reviews_for_each_day["review_id"][indices[k]])
            else:
                reviews_by_date_for_pos.append(reviews_for_each_day["review_id"][indices[k]])

        if len(reviews_by_date_for_pos) > 3:
            remove_reviews.extend(reviews_by_date_for_pos)

        if len(reviews_by_date_for_neg) > 3:
            remove_reviews.extend(reviews_by_date_for_neg)


In [17]:
len(remove_reviews)

4

In [18]:
print(remove_reviews)

['RRB9C05HDOD4O', 'R8MEA6IGAHO0B', 'R31LOQ8JGLPRLK', 'R2Y0MM9YE6OP3P']



##  Similar reviews posted in the same time interval

In [19]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
tfidf_vectorizer = TfidfVectorizer()

In [21]:
dataset.reset_index()
dataset.set_index("review_id")
dataset.sort_values("timestamp",inplace=True)

In [22]:
def OnlyStopwords(str):
    words = nltk.word_tokenize(str)
    words = [word for word in words if word not in stopwords.words("english")]
    if(len(words)==0):
        return True
    return False

In [23]:
from nltk.corpus import wordnet

remove_reviews = []
indices = []
for i in range(len(dataset)):
    
    reviews = [str(dataset["review_body"][i])]
    
    try:
        tfidf_vectorizer.fit_transform(reviews)
    except:
        # reviews with one word and with no dictionary meaning will be invalid
        # e.g- ["c","O.K."]
        remove_reviews.append(dataset["review_id"][i]) 
        continue
        
    Time = dataset["timestamp"][i]
    # timestamp of the review that will be compared
    
    for j in range(i+1,len(dataset)):
            
        indices.append(dataset["review_id"][j])
        
        if(dataset["timestamp"][j]-Time <= 1800):
            # reviews written in 30 min of intervals will be checked for same pattern
            reviews.append(str(dataset["review_body"][j]))
        else:
            break
    
    tfidf_matrix = tfidf_vectorizer.fit_transform(reviews)
    
    #creates TF-IDF Model
    tfidf_list = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix).tolist()
    # Creates matrix based on document similarity
         
    # To check similarity b/w 2 reviews
    i_appended = False
    for k in range(1,len(tfidf_list[0])):
        #print(tfidf_list[0][k],i+k)
        
        if(tfidf_list[0][k]>0.6):
            # 0.6 is defind for the simmilarity level
            
            remove_reviews.append(dataset["review_id"][i+k])
            # i+k is to get the review id of the review
            
            if(not i_appended):
                remove_reviews.append(dataset["review_id"][i]) 
                i_appended = True
 

In [24]:
len(remove_reviews)

4

In [25]:
print(remove_reviews)

['R2AT2426ZHFUHH', 'R2AT2426ZHFUHH', 'R8MEA6IGAHO0B', 'R8MEA6IGAHO0B']


##  Reviews in which Reviewer using arming tone to by the product (Action)

In [26]:
#7. Removing reviews with no. of verbs > no. of nouns

for i in range(len(dataset)):
    #iterate the whole dataset
    
    words = nltk.word_tokenize(str(dataset["review_body"][i]))
    #storing the words from the reviews into the list
    
    tagged_words = nltk.pos_tag(words)
    # returns list of tuples of words along with their parts of speech
    
    nouns_count = 0
    verbs_count = 0
    
    for j in range(len(tagged_words)):
        #iterate through all the words

        if(tagged_words[j][1].startswith("NN")):
            nouns_count+=1
            #counts the no. of nouns in the review

        if(tagged_words[j][1].startswith("VB")):
            verbs_count+=1
            #counts the no. of verbs in the review

    if(verbs_count>nouns_count):
        #comparing the no. of verbs and nouns
        remove_reviews.append(dataset["review_id"][i])
        #storing the review to be removed 

In [27]:
print(remove_reviews)

['R2AT2426ZHFUHH', 'R2AT2426ZHFUHH', 'R8MEA6IGAHO0B', 'R8MEA6IGAHO0B', 'R8MEA6IGAHO0B', 'R31LOQ8JGLPRLK', 'R26I2RI1GFV8QG', 'R2AT2426ZHFUHH', 'R8MEA6IGAHO0B', 'R8MEA6IGAHO0B', 'R31LOQ8JGLPRLK', 'RRB9C05HDOD4O', 'R26I2RI1GFV8QG', 'R2AT2426ZHFUHH', 'RLIL6S9OGM3YA', 'RT12RQFU0V2AX', 'R1IG9KI4EWNE4W', 'R260G1C72KG6KC', 'R31SFE3UEUA0EY', 'R174DGBVW0UWAJ', 'R2074W1A28UUU6', 'R3B3E2E0123J3F', 'R3VUB98IS1VKQC', 'R18TG2USOSVOC3', 'R2DWIADFBQ3IYG', 'R2VMBHZ9XTC0GQ', 'R27GI77VF2P6WF', 'R35NN211X0RGH9', 'R18MK5UOS41XJN', 'R2K2WK38XR5FKZ', 'R3FBUNWANUJRTU', 'RGAH8RF1K8QUI', 'R3KOQ03SYO1FM2', 'R2TMCO418IRILU']


In [28]:
len(remove_reviews)

34

## Reviews in which reviewer is writing his own story

In [29]:

for i in range(len(dataset)):
    #iterate through all the reviews
    
    dataset["review_body"][i] = str(dataset["review_body"][i]).lower()
    # converting each characters to its lower cases
    
    words = nltk.word_tokenize(dataset["review_body"][i])
    # storing the list of words for each reviews
    
    sentence = nltk.sent_tokenize(dataset["review_body"][i])
    # storing the list of sentences for each reviews
    
    count=0
    if(len(sentence)>4):
        # Checking only those reviews which have atleast 5 sentences.
        
        for j in range(len(words)):
            #iterating through all the reviews
            
            if(words[j]=="i" or words[j]=="we"):
                #counting personal pronouns
                count+=1
                
        if(count > len(sentence)/2):
            #reviews with number of personal pronouns used greater than half the no. of sentences.
            remove_reviews.append(dataset["review_id"][i])


/home/noel/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [30]:
len(remove_reviews)

41

In [31]:
print(remove_reviews)

['R2AT2426ZHFUHH', 'R2AT2426ZHFUHH', 'R8MEA6IGAHO0B', 'R8MEA6IGAHO0B', 'R8MEA6IGAHO0B', 'R31LOQ8JGLPRLK', 'R26I2RI1GFV8QG', 'R2AT2426ZHFUHH', 'R8MEA6IGAHO0B', 'R8MEA6IGAHO0B', 'R31LOQ8JGLPRLK', 'RRB9C05HDOD4O', 'R26I2RI1GFV8QG', 'R2AT2426ZHFUHH', 'RLIL6S9OGM3YA', 'RT12RQFU0V2AX', 'R1IG9KI4EWNE4W', 'R260G1C72KG6KC', 'R31SFE3UEUA0EY', 'R174DGBVW0UWAJ', 'R2074W1A28UUU6', 'R3B3E2E0123J3F', 'R3VUB98IS1VKQC', 'R18TG2USOSVOC3', 'R2DWIADFBQ3IYG', 'R2VMBHZ9XTC0GQ', 'R27GI77VF2P6WF', 'R35NN211X0RGH9', 'R18MK5UOS41XJN', 'R2K2WK38XR5FKZ', 'R3FBUNWANUJRTU', 'RGAH8RF1K8QUI', 'R3KOQ03SYO1FM2', 'R2TMCO418IRILU', 'R2K0YQPV5W2SJ1', 'R3EQYRGIIJQBHT', 'R260G1C72KG6KC', 'R2074W1A28UUU6', 'R3TKTB8TPDMW12', 'R2JK5Y8D5MXAGP', 'R3KOQ03SYO1FM2']


#  Meaningless Texts in reviews using LSA

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import nltk
import re
from nltk.corpus import stopwords 
import collections
from nltk.corpus import wordnet

In [33]:
dataset.set_index("review_id",inplace=True)

In [34]:
# Latent symantic analysis
# it will analyse all reviews and determine all reviews belong to the same concept
def LSA(text):
    #text is list of reviews of same product
    
    
    # Created TF-IDF Model
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(text)
    
    # Created SVD(Singular Value Decomposition)
    lsa = TruncatedSVD(n_components = 1,n_iter = 100)
    lsa.fit(X)
    
    terms = vectorizer.get_feature_names()
    concept_words={}

    for j,comp in enumerate(lsa.components_):
        componentTerms = zip(terms,comp)
        sortedTerms = sorted(componentTerms,key=lambda x:x[1],reverse=True)
        sortedTerms = sortedTerms[:10]
        concept_words[str(j)] = sortedTerms
     
    sentence_scores = []
    for key in concept_words.keys():
        for sentence in text:
            words = nltk.word_tokenize(sentence)
            scores = 0
            for word in words:
                for word_with_scores in concept_words[key]:
                    if word == word_with_scores[0]:
                        scores += word_with_scores[1]
            sentence_scores.append(scores)
    return sentence_scores

In [35]:
import nltk
# nltk.download('omw-1.4')

product_df = dataset.groupby("product_id")
#grouping dataset by product_id

unique_products = dataset["product_id"].unique()
#stores list of all product id

no_products = len(unique_products.tolist())
#no. of products

remove_reviews = []
#store review_id that are fake 
    
for i in range(no_products):
    #iterate through all product reviews 
    
    df = product_df.get_group(unique_products[i])
    #dataframe of a single product
    
    unique_reviews = df.index.tolist()
    #list of review_id of reviews of same product
    
    no_reviews = len(unique_reviews)   
    #no. of reviews of same product
    
    count = no_reviews 
    #count is no. of reviews that can be analysed
    
    reviews = []
    #list of review texts
    
    review_id = []
    #list of review texts
    
    for j in range(no_reviews):
        #iterate through all reviews
        
        text = str(df.loc[unique_reviews[j]]["review_body"])
        # text of a review 
        
        #cleaning the text
        text = re.sub(r"\W"," ",text)             
        text = re.sub(r"\d"," ",text)             
        text = re.sub(r"\s+[a-z]\s+"," ",text)    
        text = re.sub(r"^[a-z]\s+"," ",text)    
        text = re.sub(r"\s+[a-z]$"," ",text)    
        text = re.sub(r"\s+"," ",text)
        
        words = nltk.word_tokenize(text)
        #text into word list
        
        if(len(words)==1):
        #if only one word in text review
            
            if(len(text) <=1 or not wordnet.synsets(text) ):
            #if word is having only one character or invalid english word
                
                remove_reviews.append(unique_reviews[j])
                #append this review as fake
                
                count-=1
                #review left to be analysed will be decrease by 1
                
                continue
                #check for next review 
                
        elif(len(words)==0):
        #if no words
            
            remove_reviews.append(unique_reviews[j])
            #append this review as fake
            
            count-=1
            #review left to be analysed will be decrease by 1
            
            continue
            #check for next review
        
        review_id.append(unique_reviews[j])
        #valid: append review_id to review_id list for analysis
        
        reviews.append(text)
        #valid: append review_body to reviews list for analysis
        
    ###########################################################################################
    if(count<=0):
        #if no reviews left to analyse 

        continue
        #check for next
        
    if(count==1): 
        #if only one review is left to analyse
        
        #check concept between product title and the review
        
        text = [text,str(df.loc[review_id[0]]["product_title"])] 
        #add product_title and review to the list 
        
        sentence_scores = LSA(text) 
        #call LSA method to get the score
        
        if(sentence_scores[0]==0): 
        #if review score is 0, then it's fake
            remove_reviews.append(review_id[0])
        continue
    
    #list of scores of reviews of same product
    sentence_scores = LSA(reviews)
            
    for j in range(len(sentence_scores)):
        #iterating through all the scores
        
        if(sentence_scores[j]==0.00):
            # if score is 0, it's fake
            remove_reviews.append(review_id[j])

/home/noel/anaconda3/envs/myenv/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


/home/noel/anaconda3/envs/myenv/lib/python3.7/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var


In [36]:
len(remove_reviews)

14

In [37]:
print(remove_reviews)

['RHK9SLC4KVI7E', 'RBKUGHHNXVYG0', 'R1HZRM3VPO0TFQ', 'R174E63UVEWXYD', 'R2JTHBK0R10ZW4', 'R2K2WK38XR5FKZ', 'R802NI63NKL9V', 'R1T578QAZDASOQ', 'R34EZZ68VYPHO0', 'R2J68T8Y2ZWUSP', 'R36A0KHN48CHBP', 'R2RWA9DSTPVRCI', 'R250PR8VJUZ62F', 'R104MY31OKNGXH']


# Removing Fake Reviews

In [38]:
dataset.drop(remove_reviews,inplace=True)

In [39]:
dataset = dataset.set_index("IP Address")

In [40]:
dataset.drop(remove_ip,inplace=True)

In [41]:
dataset.to_csv("real_reviews.csv",sep="\t")